In [75]:
from selenium import webdriver
import pandas as pd
import string

In [77]:
path = "****" # please install the chromedriver according to your chrome version from https://chromedriver.chromium.org/downloads and add the path to this
driver = webdriver.Chrome(path) #linking chrome driver from the file directory

complete_med_info = [] # list that will be appended with all the data

def get_data(data):
   driver.get(f'https://www.pbs.gov.au/browse/medicine-listing?initial={data}') #parent page directory 

   #scraping drug type (community , general, public etc) from the parent page:

   scrap_drug_type = driver.find_elements_by_class_name('drugtype')
   for drug_types in scrap_drug_type:
      drug_type = drug_types.text
   
   #going through pages one by one below:

   main_page = driver.find_elements_by_css_selector('#browseby a') #scraping from the parent page 
   second_nested_page_links = [] # contains the links that will guide to individual medicine pages(eg - 'abacaviar' (first medicine in page 'A'))
   
   for link in main_page:
      second_nested_page_links.append(link.get_attribute("href"))

   third_nested_page_links = [] # contains the links that will guide to separe sub-medicines (eg: sublist of abacaviar)

   for link in second_nested_page_links:
      driver.get(link)
      scrap_third_page_links = driver.find_elements_by_class_name("view-all") 
      
      for link in scrap_third_page_links:
         third_nested_page_links.append(link.get_attribute("href"))
   
   med_info = [] #contains the final page information (brands, strength etc) of particular medicines
   for web_page in third_nested_page_links:
      driver.get(web_page)
      scrap_third_page = driver.find_elements_by_id('medicine-item')
      
      for x in scrap_third_page:
         med_name = x.find_element_by_class_name('drug-name').text
         med_strength =x.find_element_by_class_name('form-strength').text
         available_brand_name_lists= x.find_elements_by_css_selector('.align-left a')
         product_codes = x.find_elements_by_css_selector('.align-top a')
         available_brands_names_list = []
         
         for item_codes in product_codes:
            product_code = item_codes.text
            
         for i in available_brand_name_lists:
            available_brand_name_texts = i.text
            available_brands_names_list.append(available_brand_name_texts)
         available_brands = available_brands_names_list[3:]
         
         available_brand = str(available_brands)[1:-1].replace("'", "") #replacing extra brackets and commas so it can be properly converted to dataframe
         
         #creating a dictionary that will hold all the medicine information
         med_info_dict = { 'Medicine_Name' : med_name,
         'Drug_Type' : drug_type,
         'Product_code' : product_code,
         'Medicine_strength' : med_strength,
         'Available_Brands' : available_brand}
         

         med_info.append(med_info_dict)
         df = pd.DataFrame(med_info)
         
       
   return pd.DataFrame(df)

pages_from_A_Z = list(string.ascii_lowercase) #creating a list A-Z for scraping the pages alphabetically
for i in pages_from_A_Z: #using try except methods as not all the letters have medicine starting name
   try:
      complete_med_info.append(get_data(i))
   except:
      continue

driver.quit()


In [84]:
combined = pd.concat(complete_med_info) # joining all the individual medicine info (categorically A-Z)
combined.index += 1
df_with_reset_index = combined.reset_index()
combined.index += 1

df_with_reset_index.to_csv('medicine_info.csv',index=False)